# Action-Conditioned 3D Human Motion Synthesis with Transformer VAE (ACTOR)

ACTOR learns an action-aware latent representation for human motions by training a generative variational autoencoder (VAE). By sampling from this latent space and querying a certain duration through a series of positional encodings, ACTOR synthesizes variable-length motion sequences conditioned on a categorical action. Specifically, a Transformer-based architecture is designed, for encoding and decoding a sequence of parametric SMPL human body models estimated from action recognition datasets. 

## Dataset

To get the pre-process the dataset, please refer to the this [Github repository](https://github.com/Mathux/ACTOR) and agree to the license. There following code shows examples from `HumanAct12` dataset.

In [1]:
# Set data path
datapath = "F://research/ACTOR/data/HumanAct12Poses/"

In [2]:
# change the working directory to the source code folder
import os
os.chdir("../../../../src")
os.getcwd()

'E:\\researches\\GenMotion\\src'

## Training

In [3]:
import torch

In [4]:
from algorithm.action_conditioned.params import HumanAct12Params
from algorithm.action_conditioned.data_utils import get_datasets

from algorithm.action_conditioned.utils.tensors import collate

In [5]:
# load parameters
parameters = vars(HumanAct12Params())

In [6]:
# get datasets
datasets = get_datasets(datapath, parameters)
print("dataset length: ", {key: len(val) for key, val in datasets.items()})

dataset length:  {'train': 1190, 'test': 1190}


In [7]:
# load model
assert parameters["modeltype"] == 'cvae'
assert parameters["archiname"] == "transformer"

from algorithm.action_conditioned.models.architectures.transformer import Encoder_TRANSFORMER, Decoder_TRANSFORMER
from algorithm.action_conditioned.models.modeltype.cvae import CVAE

encoder = Encoder_TRANSFORMER(**parameters)
decoder = Decoder_TRANSFORMER(**parameters)

parameters["outputxyz"] = "rcxyz" in parameters["lambdas"]

In [8]:
model = CVAE(encoder, decoder, **parameters).to(parameters["device"])

In [9]:
# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=parameters["lr"])
print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))
# print("Training model..")

Total params: 14.83M


In [10]:
dataset = datasets["train"]
train_iterator = torch.utils.data.DataLoader(dataset, batch_size=parameters["batch_size"], 
                                             shuffle=True, num_workers=8, collate_fn=collate)

In [11]:
from algorithm.action_conditioned.trainer import train

In [12]:
model.device

device(type='cuda')

In [ ]:
dict_loss = train(model, optimizer, train_iterator, model.device)

Computing batch: 16it [00:07,  2.37it/s]